In [64]:
from os.path import exists as exists
from os import listdir as listdir
import pandas as pd
import networkx as nx

In [97]:
RULES = [1,2]
ENLACES = [32,16,8,4,2,1]
DEGRADATION_REPETITIONS = [[1],list(range(1,11))]
DEGRADATION_TYPE = ["ataques", "fallos"]
D = 0
DEGRADATION = DEGRADATION_TYPE[D]
REPETITIONS = DEGRADATION_REPETITIONS[D]
PATH_GRAPHS = "graphs-degradation\\"
PATH_DEGRADATION = "data_critical_threshold\\attr\\attr_" + DEGRADATION + "_"
PATH_MATTR = "data_critical_threshold\\mattr\\mattr_" + DEGRADATION + "_"
PATH_PLOT_FILES = "plot_files\\mattr\\mattr_"

In [98]:
def save_csv(data_df, path_file):
    print("Save: ", path_file)
    data_df.to_csv(path_file, sep=',',encoding='utf-8')

# Calcula la distribucion de componentes y attr

In [99]:
def compute_attr(cluster_list, n):
    num_paths = [i*(i-1) for i in cluster_list if i>1]
    if len(num_paths) == 0:
        return 0
    else:
        return sum(num_paths) / (n*(n-1))       

In [100]:
def cluster_distribution(G):
    return [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]

In [101]:
def degradate_network(G, removed_nodes):
    H = G.copy()
    num_nodes = [2500]
    fc = [1]
    clusters = [[2500]]
    attr = [1]
    for i in range(100):
        rnodes = removed_nodes[i*25:(i*25+25)]
        H.remove_nodes_from(rnodes)
        remain_nodes = len(H.nodes())
        num_nodes.append(remain_nodes)
        fc.append(remain_nodes / 2500)
        cluster_list = cluster_distribution(H)
        clusters.append(cluster_list)
        a2tr = compute_attr(cluster_list, remain_nodes)
        attr.append(a2tr)
    data = {'num_nodes': num_nodes, 'fc': fc, 'clusters': clusters, 'attr': attr}
    df_dist =pd.DataFrame.from_dict(data)
    return df_dist
    #print(df_dist.iloc[[30]])

In [102]:
for r in RULES:
    for enlace in ENLACES:
        for exp in range(1,11):
            p_dir = PATH_GRAPHS + "r" + str(r) + "\\enlaceD" + str(enlace) + "\\experimento" + str(exp)
            files = listdir(p_dir)
            name_graph = [x for x in files if ".adjlist" in x][0]
            p_graph =  p_dir + '\\' + name_graph
            G = nx.read_adjlist(p_graph)
            for repetition in REPETITIONS:
                p_attr = PATH_DEGRADATION + "r" + str(r) + "_enlaceD" + str(enlace) + "_experimento" + str(exp) + "_repetition" + str(repetition) + ".csv"
                p_degradationData = PATH_GRAPHS + "r" + str(r) + "\\enlaceD" + str(enlace) + "\\experimento" + str(exp) + "\\" + DEGRADATION + "\\" + str(repetition) + "\\"
                if not exists (p_attr):
                    print(p_attr)
                    print(p_degradationData)
                    if D == 0:
                        p_removed = p_degradationData + "AttackSequence.txt"
                    else:
                        p_removed = p_degradationData + "FailureSequence.txt"
                    # reading the file 
                    file_removed = open(p_removed, "r") 
                    data = file_removed.read() 
                    removed_nodes = data.split("\n")
                    rnodes = [i for i in removed_nodes if i]
                    df_attr = degradate_network(G, rnodes)
                    save_csv(df_attr, p_attr)
                else:
                    print("Ya existe ", p_attr)

Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento1_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento2_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento3_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento4_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento5_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento6_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento7_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento8_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento9_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r1_enlaceD32_experimento10_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataq

Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD8_experimento9_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD8_experimento10_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento1_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento2_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento3_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento4_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento5_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento6_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento7_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enlaceD4_experimento8_repetition1.csv
Ya existe  data_critical_threshold\attr\attr_ataques_r2_enl

# Calcula Miu ATTR

In [103]:
p_mattr = PATH_PLOT_FILES + DEGRADATION + '.csv'
if not exists(p_mattr):
    print(p_mattr)
    dic_mattr = dict()
    for r in RULES:
        long_enlaces = list()
        experi = list()
        rep = list()
        miu_attr = list()
        for enlace in ENLACES:
            for exp in range(1,11):
                for repetition in REPETITIONS:
                    p_attr = PATH_DEGRADATION + "r" + str(r) + "_enlaceD" + str(enlace) + "_experimento" + str(exp) + "_repetition" + str(repetition) + ".csv"
                    if exists (p_attr):
                        #print(p_attr)
                        df_attr = pd.read_csv(p_attr)
                        list_attr = list(df_attr['attr'])
                        miu_attr.append(sum(list_attr) / len(list_attr)) 
                        long_enlaces.append(enlace)
                        experi.append(exp)
                        rep.append(repetition)
        data = {'enlace': long_enlaces, 'experimento': experi, 'repetition': rep, 'mattr': miu_attr}
        df_mattr = pd.DataFrame.from_dict(data)
        #print(df_mattr)
        dic_mattr['mean_r' + str(r) + '_mattr'] = df_mattr.groupby('enlace')['mattr'].mean()
        dic_mattr['std_r' + str(r) + '_mattr'] = df_mattr.groupby('enlace')['mattr'].std()
    pd_results = pd.DataFrame(dic_mattr)
    pd_results.index.name = 'enlace'
    #pd_results = pd_results.fillna(0)
    pd_results = pd_results.sort_values(by=['enlace'],ascending=False)
    save_csv(pd_results, p_mattr)

plot_files\mattr\mattr_ataques.csv
        mean_r1_mattr  std_r1_mattr  mean_r2_mattr  std_r2_mattr
enlace                                                          
32           0.445352      0.007902       0.446516      0.010479
16           0.464553      0.002626       0.446553      0.001852
8            0.443330      0.004697       0.441189      0.005634
4            0.413887      0.008253       0.450320      0.006217
2            0.386277      0.011954       0.465865      0.008389
1            0.410767      0.053282       0.474208      0.003914
Save:  plot_files\mattr\mattr_ataques.csv
